# Test environment 
Check whether you can run the cells below without errors

## Try to import the libraries

In [ ]:
from math import pi

import casadi
import numpy as np
from matplotlib import pyplot as plt
from pinocchio import casadi as cpin
from ipywidgets import interact
from example_robot_data import load
from pinocchio.visualize import MeshcatVisualizer
import pandas

## Say hello to Talos

In [ ]:
talos = load("talos")
viz = MeshcatVisualizer(talos.model, talos.collision_model, talos.visual_model)
viz.initViewer(loadModel=True)
viz.viewer.jupyter_cell()

# click on the url below to open it in a standalone browser window

In [ ]:
q0 = talos.q0
viz.display(q0)

### See whether Talos can wave back

In [ ]:
@interact(shoulder=(-0.5, 0.5, 0.01))
def say_hi(shoulder=0):
    q0[30] = pi - shoulder
    viz.display(q0)

In [ ]:
differentiable_talos = cpin.Model(talos.model)
differentiable_data = differentiable_talos.createData()
cpin.forwardKinematics(differentiable_talos, differentiable_data, casadi.SX(q0))

## Warm up for the course by solving some optimization problems

Rosenbrock problem:
$$
\min_{x,y} \quad (1-x)^2 + (y - x^2)^2
$$

### Formulate the problem using CasADi

In [ ]:
opti = casadi.Opti()

x = opti.variable()
y = opti.variable()

opti.minimize((1-x)**2 + (y-x**2)**2)

# Choose IPOPT as the solver and solve
opti.solver('ipopt')
sol = opti.solve()

# Get the optimal solution values
x_opt = sol.value(x)
y_opt = sol.value(y)
print("Optimal solution: x =", x_opt, "y =", y_opt)


### Plot the results

In [ ]:
# Create a grid to evaluate the function for contour plotting
x_vals = np.linspace(-1, 2, 400)
y_vals = np.linspace(-1, 3, 400)
X, Y = np.meshgrid(x_vals, y_vals)
Z = (1 - X)**2 + (Y - X**2)**2

# Plot the contours of the function
plt.figure(figsize=(8, 6))
contour_levels = np.logspace(-2, 3, 20)  # logarithmically spaced contour levels
CS = plt.contour(X, Y, Z, levels=contour_levels, cmap='viridis')
plt.clabel(CS, inline=1, fontsize=10)
plt.plot(x_opt, y_opt, 'r*', markersize=15, label='Optimal solution')
plt.title('Contour Plot and Optimal Solution')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()